In [25]:
import torch
# import keras
import numpy
import pandas
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas as pd
from sklearn import preprocessing
from string import ascii_letters
import seaborn as sns
import matplotlib.pyplot as plt
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
# from torchvision import dfs, transforms
import torch.nn.functional as F


In [26]:
# drive.mount('/content/drive')

df = pd.read_csv('train_dataset.csv')
df_test = pd.read_csv('test_dataset.csv')
df_mobile = pd.read_csv('mobile_plans.csv')
df.head
df.age_group.mode()


0    30-40
dtype: object

In [27]:
df.age_group.isna().sum()

135

In [28]:
S = {'FEMALE':1,'MALE':0}
df = df.replace({"gender": S})
df_test = df_test.replace({'gender': S})
P = {'PKG1':1,"PKG2":2,'PKG3':3,'PKG4':4,'PKG5':5,'PKG6':6,'PKG7':7,'PKG8':8}
df = df.replace({"next_month_plan": P})
df_test = df_test.replace({'next_month_plan': P})

In [29]:
df.describe()

,primary_identifier,gender,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,dusage_max,dusage_avg,dusage_days,dusage_stddev,...,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count,next_month_plan
count,1.050000e+04,10365.000000,10500.000000,10500.000000,10500.000000,10500.000000,10500.000000,9642.000000,10500.000000,10500.000000,...,10500.000000,10500.000000,10500.000000,7187.000000,10500.000000,10500.000000,10500.000000,6667.000000,6667.000000,10500.000000
mean,5.531286e+06,0.283261,91.984381,2083.923835,12813.022667,72.723046,1440.667844,492.631079,24.791524,330.034959,...,14.700308,0.024573,8.970667,6.559910,3.493543,0.052857,0.246762,518.560072,1.889156,3.059238
std,2.597973e+06,0.450604,71.888784,1470.550224,15873.353992,168.971572,1846.417470,552.332657,9.677004,403.594192,...,22.039382,0.360990,9.454676,7.527537,5.231989,0.236587,0.500846,622.863583,1.224138,2.057261
min,1.000454e+06,0.000000,3.000000,-11446.340000,0.000000,0.000000,0.000000,0.000042,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,3.286428e+06,0.000000,22.000000,1036.270000,1939.000000,0.000000,367.013343,123.664867,25.000000,89.722583,...,0.000000,0.000000,0.000000,2.397829,0.000000,0.000000,0.000000,200.000000,1.000000,1.000000
50%,5.538684e+06,0.000000,82.000000,1783.845000,7800.500000,2.385201,977.321670,328.181079,30.000000,232.348678,...,6.858333,0.000000,6.000000,4.419298,1.614136,0.000000,0.000000,360.000000,2.000000,3.000000
75%,7.799776e+06,1.000000,149.000000,2700.592500,17836.000000,64.012977,1840.254781,665.463278,30.000000,424.423794,...,21.087500,0.000000,16.000000,7.851111,5.016638,0.000000,0.000000,650.000000,2.000000,5.000000
max,9.999195e+06,1.000000,314.000000,19007.890000,176410.000000,3501.540916,30513.071629,7386.100749,30.000000,6913.568069,...,477.300000,25.566667,30.000000,107.998148,90.944793,3.000000,3.000000,19450.000000,24.000000,8.000000


In [30]:
df.isna().sum()

primary_identifier                0
device_type                      23
device_category                  30
gender                          135
district_name                     0
age_group                       135
network_stay                      0
average_monthly_bill_amount       0
dusage_sum                        0
dusage_min                        0
dusage_max                        0
dusage_avg                      858
dusage_days                       0
dusage_stddev                     0
vusage_onnet_sum                  0
vusage_onnet_max                  0
vusage_onnet_min                  0
vusage_onnet_days                 0
vusage_onnet_avg               8672
vusage_onnet_stddev               0
vusage_offnet_sum                 0
vusage_offnet_max                 0
vusage_offnet_min                 0
vusage_offnet_days                0
vusage_offnet_avg              3313
vusage_offnet_stddev              0
number_of_fixed_bb_accounts       0
number_of_iptv_accounts     

In [31]:
max_dt = df.device_type.mode()
max_dt[0]

'4G'

In [32]:
#train
max_gender = df.gender.mode()
df.gender = df.gender.replace(np.nan,max_gender[0])

max_dt = df.device_type.mode()
df.device_type = df.device_type.replace(np.nan,max_dt[0])

max_dc = df.device_category.mode()
df.device_category = df.device_category.replace(np.nan,max_dc[0])

max_age = df.age_group.mode()
df.age_group = df.age_group.replace(np.nan,max_age[0])

#test
max_gender_t = df_test.gender.mode()
df_test.gender = df_test.gender.replace(np.nan,max_gender_t[0])

max_dt_t = df_test.device_type.mode()
df_test.device_type = df_test.device_type.replace(np.nan,max_dt_t[0])

max_dc_t = df_test.device_category.mode()
df_test.device_category = df_test.device_category.replace(np.nan,max_dc_t[0])

max_age_t = df_test.age_group.mode()
df_test.age_group = df_test.age_group.replace(np.nan,max_age_t[0])

In [33]:
mean_addot = df.add_on_tot_rental.mean()
mean_addot

518.5600719964002

In [34]:
df.dusage_avg=df.dusage_avg.replace(np.nan,0)     
df.vusage_onnet_avg=df.vusage_onnet_avg.replace(np.nan,0)      
df.vusage_offnet_avg=df.vusage_offnet_avg.replace(np.nan,0) 

df_test.dusage_avg=df.dusage_avg.replace(np.nan,0)     
df_test.vusage_onnet_avg=df.vusage_onnet_avg.replace(np.nan,0)      
df_test.vusage_offnet_avg=df.vusage_offnet_avg.replace(np.nan,0) 

mean_addot = df.add_on_tot_rental.mean()
df.add_on_tot_rental = df.add_on_tot_rental.replace(np.nan,mean_addot)

mean_addoc = df.add_on_count.mean()
df.add_on_count = df.add_on_count.replace(np.nan,mean_addoc)

mean_addot_t = df_test.add_on_tot_rental.mean()
df_test.add_on_tot_rental = df_test.add_on_tot_rental.replace(np.nan,mean_addot_t)

mean_addoc_t = df_test.add_on_count.mean()
df_test.add_on_count = df_test.add_on_count.replace(np.nan,mean_addoc_t)
                    

In [35]:
df.isna().sum()

primary_identifier             0
device_type                    0
device_category                0
gender                         0
district_name                  0
age_group                      0
network_stay                   0
average_monthly_bill_amount    0
dusage_sum                     0
dusage_min                     0
dusage_max                     0
dusage_avg                     0
dusage_days                    0
dusage_stddev                  0
vusage_onnet_sum               0
vusage_onnet_max               0
vusage_onnet_min               0
vusage_onnet_days              0
vusage_onnet_avg               0
vusage_onnet_stddev            0
vusage_offnet_sum              0
vusage_offnet_max              0
vusage_offnet_min              0
vusage_offnet_days             0
vusage_offnet_avg              0
vusage_offnet_stddev           0
number_of_fixed_bb_accounts    0
number_of_iptv_accounts        0
add_on_tot_rental              0
add_on_count                   0
next_month

In [36]:
df_test.isna().sum()

primary_identifier             0
device_type                    0
device_category                0
gender                         0
district_name                  0
age_group                      0
network_stay                   0
average_monthly_bill_amount    0
dusage_sum                     0
dusage_min                     0
dusage_max                     0
dusage_avg                     0
dusage_days                    0
dusage_stddev                  0
vusage_onnet_sum               0
vusage_onnet_max               0
vusage_onnet_min               0
vusage_onnet_days              0
vusage_onnet_avg               0
vusage_onnet_stddev            0
vusage_offnet_sum              0
vusage_offnet_max              0
vusage_offnet_min              0
vusage_offnet_days             0
vusage_offnet_avg              0
vusage_offnet_stddev           0
number_of_fixed_bb_accounts    0
number_of_iptv_accounts        0
add_on_tot_rental              0
add_on_count                   0
dtype: int

In [37]:
df_test.gender

0       0.0
1       0.0
2       0.0
3       1.0
4       0.0
       ... 
4495    0.0
4496    0.0
4497    1.0
4498    0.0
4499    0.0
Name: gender, Length: 4500, dtype: float64

In [38]:
df.isnull().any()

primary_identifier             False
device_type                    False
device_category                False
gender                         False
district_name                  False
age_group                      False
network_stay                   False
average_monthly_bill_amount    False
dusage_sum                     False
dusage_min                     False
dusage_max                     False
dusage_avg                     False
dusage_days                    False
dusage_stddev                  False
vusage_onnet_sum               False
vusage_onnet_max               False
vusage_onnet_min               False
vusage_onnet_days              False
vusage_onnet_avg               False
vusage_onnet_stddev            False
vusage_offnet_sum              False
vusage_offnet_max              False
vusage_offnet_min              False
vusage_offnet_days             False
vusage_offnet_avg              False
vusage_offnet_stddev           False
number_of_fixed_bb_accounts    False
n

In [39]:
dusage_avg = [i for i in range(0,7500,100)]
df['dusage_avg'] =pd.cut(df['dusage_avg'],dusage_avg,labels=dusage_avg[1:],retbins=False)
df_test['dusage_avg']=pd.cut(df_test['dusage_avg'],dusage_avg,labels=dusage_avg[1:],retbins=False)

In [40]:
dusage_sum = [i for i in range(0,178000,1000)]
df['dusage_sum']=pd.cut(df['dusage_sum'],dusage_sum,labels=dusage_sum[1:],retbins=False)
dusage_min = [i for i in range(0,3700,100)]
df['dusage_min']=pd.cut(df['dusage_min'],dusage_min,labels=dusage_min[1:],retbins=False)
dusage_max = [i for i in range(0,30700,100)]
df['dusage_max']=pd.cut(df['dusage_max'],dusage_max,labels=dusage_max[1:],retbins=False)

In [41]:
dusage_sum = [i for i in range(0,178000,1000)]
df_test['dusage_sum']=pd.cut(df_test['dusage_sum'],dusage_sum,labels=dusage_sum[1:],retbins=False)
dusage_min = [i for i in range(0,3700,100)]
df_test['dusage_min']=pd.cut(df_test['dusage_min'],dusage_min,labels=dusage_min[1:],retbins=False)
dusage_max = [i for i in range(0,30700,100)]
df_test['dusage_max']=pd.cut(df_test['dusage_max'],dusage_max,labels=dusage_max[1:],retbins=False)

In [42]:
df.average_monthly_bill_amount

0         761.62
1        2482.52
2        1565.55
3        2397.49
4        2501.75
          ...   
10495    2088.13
10496    2053.50
10497    2827.33
10498     645.43
10499     498.20
Name: average_monthly_bill_amount, Length: 10500, dtype: float64

In [43]:
df

,primary_identifier,device_type,device_category,gender,district_name,age_group,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,...,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count,next_month_plan
0,6875768,4G,Smartphone,0.0,Kegalle,40-50,219,761.62,7000,100,...,152.866667,0.0,27.0,25.099383,34.759221,0.0,1.0,518.560072,1.889156,2
1,6406277,4G,Smartphone,0.0,Galle,40-50,36,2482.52,29000,100,...,35.433333,0.0,15.0,5.071111,6.779644,0.0,0.0,600.000000,2.000000,1
2,3563570,4G,Smartphone,0.0,Kandy,30-40,186,1565.55,19000,100,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.000000,6
3,1853666,4G,Smartphone,1.0,Colombo,40-50,24,2397.49,92000,100,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,1110.000000,2.000000,4
4,2794331,4G,Smartphone,1.0,Kandy,50-60,37,2501.75,20000,100,...,87.616667,0.0,23.0,11.256522,16.224071,0.0,1.0,1350.000000,2.000000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10495,1438655,4G,Smartphone,0.0,Kalutara,40-50,64,2088.13,1000,100,...,29.866667,0.0,7.0,14.571429,8.017054,0.0,0.0,518.560072,1.889156,6
10496,7827264,4G,Smartphone,1.0,Colombo,<20,6,2053.50,24000,100,...,25.366667,0.0,3.0,10.861111,5.151029,0.0,0.0,580.000000,3.000000,1
10497,1433957,4G,Smartphone,1.0,Colombo,30-40,69,2827.33,5000,100,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,1.0,518.560072,1.889156,6
10498,8494507,4G,Smartphone,0.0,Kandy,60-70,233,645.43,3000,100,...,18.650000,0.0,18.0,5.887963,4.956822,0.0,1.0,518.560072,1.889156,1


In [44]:
df.average_monthly_bill_amount

0         761.62
1        2482.52
2        1565.55
3        2397.49
4        2501.75
          ...   
10495    2088.13
10496    2053.50
10497    2827.33
10498     645.43
10499     498.20
Name: average_monthly_bill_amount, Length: 10500, dtype: float64

In [62]:
df3 = pd.read_csv('preprocessed_all (1).csv')
df3_test = df3.iloc[-4500:]

In [63]:
x = df3[['network_stay', 'average_monthly_bill_amount', 'dusage_sum', 'dusage_min', 'dusage_max', 'dusage_avg', 'dusage_days', 'dusage_stddev', 'vusage_onnet_sum', 'vusage_onnet_max', 'vusage_onnet_min', 'vusage_onnet_days', 'vusage_onnet_avg', 'vusage_onnet_stddev', 'vusage_offnet_sum', 'vusage_offnet_max', 'vusage_offnet_min', 'vusage_offnet_days', 'vusage_offnet_avg', 'vusage_offnet_stddev', 'number_of_fixed_bb_accounts', 'number_of_iptv_accounts', 'add_on_tot_rental', 'add_on_count']].values #returns a numpy array
min_max_scaler = preprocessing.StandardScaler()
x_scaled = min_max_scaler.fit_transform(x)
df3[['network_stay', 'average_monthly_bill_amount', 'dusage_sum', 'dusage_min', 'dusage_max', 'dusage_avg', 'dusage_days', 'dusage_stddev', 'vusage_onnet_sum', 'vusage_onnet_max', 'vusage_onnet_min', 'vusage_onnet_days', 'vusage_onnet_avg', 'vusage_onnet_stddev', 'vusage_offnet_sum', 'vusage_offnet_max', 'vusage_offnet_min', 'vusage_offnet_days', 'vusage_offnet_avg', 'vusage_offnet_stddev', 'number_of_fixed_bb_accounts', 'number_of_iptv_accounts', 'add_on_tot_rental', 'add_on_count']] = pd.DataFrame(x_scaled)

In [64]:
x_t = df3_test[['network_stay', 'average_monthly_bill_amount', 'dusage_sum', 'dusage_min', 'dusage_max', 'dusage_avg', 'dusage_days', 'dusage_stddev', 'vusage_onnet_sum', 'vusage_onnet_max', 'vusage_onnet_min', 'vusage_onnet_days', 'vusage_onnet_avg', 'vusage_onnet_stddev', 'vusage_offnet_sum', 'vusage_offnet_max', 'vusage_offnet_min', 'vusage_offnet_days', 'vusage_offnet_avg', 'vusage_offnet_stddev', 'number_of_fixed_bb_accounts', 'number_of_iptv_accounts', 'add_on_tot_rental', 'add_on_count']].values #returns a numpy array
min_max_scaler = preprocessing.StandardScaler()
x_scaled = min_max_scaler.fit_transform(x_t)
df3_test[['network_stay', 'average_monthly_bill_amount', 'dusage_sum', 'dusage_min', 'dusage_max', 'dusage_avg', 'dusage_days', 'dusage_stddev', 'vusage_onnet_sum', 'vusage_onnet_max', 'vusage_onnet_min', 'vusage_onnet_days', 'vusage_onnet_avg', 'vusage_onnet_stddev', 'vusage_offnet_sum', 'vusage_offnet_max', 'vusage_offnet_min', 'vusage_offnet_days', 'vusage_offnet_avg', 'vusage_offnet_stddev', 'number_of_fixed_bb_accounts', 'number_of_iptv_accounts', 'add_on_tot_rental', 'add_on_count']] = pd.DataFrame(x_scaled)

C:\Users\dewth\anaconda3\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [65]:
df3_test.tail()

,Unnamed: 0,Unnamed: 0.1,primary_identifier,gender,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,dusage_max,dusage_avg,...,district_name_Trincomalee,district_name_Unknown,district_name_Vavuniya,age_group_20-30,age_group_30-40,age_group_40-50,age_group_50-60,age_group_60-70,age_group_<20,age_group_>70
14900,14995,4495,8830679,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,1,0,0,0
14901,14996,4496,3839728,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,0,0,0,0,0,0
14902,14997,4497,2166141,1,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0
14903,14998,4498,9426198,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,1,0,0,0,0,0
14904,14999,4499,9114395,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,1,0,0,0,0,0


In [61]:
df3_test.to_csv('1.csv')

In [47]:
df.dusage_avg=df.dusage_avg.replace(np.nan,0)     
df.vusage_onnet_avg=df.vusage_onnet_avg.replace(np.nan,0)      
df.vusage_offnet_avg=df.vusage_offnet_avg.replace(np.nan,0) 

df_test.dusage_avg=df.dusage_avg.replace(np.nan,0)     
df_test.vusage_onnet_avg=df.vusage_onnet_avg.replace(np.nan,0)      
df_test.vusage_offnet_avg=df.vusage_offnet_avg.replace(np.nan,0) 

df.dusage_sum=df.dusage_sum.replace(np.nan,0)     
df.dusage_min=df.dusage_min.replace(np.nan,0)      
df.dusage_max=df.dusage_max.replace(np.nan,0) 

df_test.dusage_sum=df_test.dusage_sum.replace(np.nan,0)     
df_test.dusage_min=df_test.dusage_min.replace(np.nan,0)      
df_test.dusage_max=df_test.dusage_max.replace(np.nan,0) 

In [48]:
from scipy.stats import zscore 

z_scores = zscore(df.average_monthly_bill_amount)
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 10000).all(axis=None)
new_df = df.average_monthly_bill_amount[filtered_entries]


print(filtered_entries)

KeyError: True

In [ ]:
df.describe()

,primary_identifier,gender,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,dusage_max,dusage_avg,dusage_days,dusage_stddev,...,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count,next_month_plan
count,1.050000e+04,10500.000000,10500.000000,10500.000000,10500.000000,10500.000000,10500.000000,10500.000000,10500.000000,10500.000000,...,10500.000000,10500.000000,10500.000000,10500.000000,10500.000000,10500.000000,10500.000000,10500.000000,10500.000000,10500.000000
mean,5.531286e+06,0.279619,0.286123,0.444282,0.070319,0.015532,0.045779,0.056112,0.826384,0.047737,...,0.030799,0.000961,0.299022,0.041576,0.038414,0.017619,0.082254,0.026661,0.038659,3.059238
std,2.597973e+06,0.448833,0.231154,0.048287,0.089902,0.045085,0.060404,0.074156,0.322567,0.058377,...,0.046175,0.014120,0.315156,0.064203,0.057529,0.078862,0.166949,0.025517,0.042409,2.057261
min,1.000454e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,3.286428e+06,0.000000,0.061093,0.409881,0.005682,0.000000,0.009836,0.000000,0.833333,0.012978,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015424,0.000000,1.000000
50%,5.538684e+06,0.000000,0.254019,0.434428,0.039773,0.000000,0.029508,0.027397,1.000000,0.033608,...,0.014369,0.000000,0.200000,0.023565,0.017749,0.000000,0.000000,0.026661,0.038659,3.000000
75%,7.799776e+06,1.000000,0.469453,0.464531,0.096591,0.000000,0.059016,0.082192,1.000000,0.061390,...,0.044181,0.000000,0.533333,0.054687,0.055161,0.000000,0.000000,0.026661,0.043478,5.000000
max,9.999195e+06,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,8.000000


In [ ]:
df.isna().sum()

primary_identifier             0
device_type                    0
device_category                0
gender                         0
district_name                  0
age_group                      0
network_stay                   0
average_monthly_bill_amount    0
dusage_sum                     0
dusage_min                     0
dusage_max                     0
dusage_avg                     0
dusage_days                    0
dusage_stddev                  0
vusage_onnet_sum               0
vusage_onnet_max               0
vusage_onnet_min               0
vusage_onnet_days              0
vusage_onnet_avg               0
vusage_onnet_stddev            0
vusage_offnet_sum              0
vusage_offnet_max              0
vusage_offnet_min              0
vusage_offnet_days             0
vusage_offnet_avg              0
vusage_offnet_stddev           0
number_of_fixed_bb_accounts    0
number_of_iptv_accounts        0
add_on_tot_rental              0
add_on_count                   0
next_month

In [ ]:
df.to_csv("train_data_null_removed.csv")
df_test.to_csv("test_data_null_removed.csv")
df.columns

Index(['primary_identifier', 'device_type', 'device_category', 'gender',
       'district_name', 'age_group', 'network_stay',
       'average_monthly_bill_amount', 'dusage_sum', 'dusage_min', 'dusage_max',
       'dusage_avg', 'dusage_days', 'dusage_stddev', 'vusage_onnet_sum',
       'vusage_onnet_max', 'vusage_onnet_min', 'vusage_onnet_days',
       'vusage_onnet_avg', 'vusage_onnet_stddev', 'vusage_offnet_sum',
       'vusage_offnet_max', 'vusage_offnet_min', 'vusage_offnet_days',
       'vusage_offnet_avg', 'vusage_offnet_stddev',
       'number_of_fixed_bb_accounts', 'number_of_iptv_accounts',
       'add_on_tot_rental', 'add_on_count', 'next_month_plan'],
      dtype='object')

In [ ]:
df2 = pd.get_dummies(df,columns = ['device_type','device_category','district_name', 'age_group'])
df2_test = pd.get_dummies(df_test,columns = ['device_type','device_category','district_name', 'age_group'])
df2_test.columns

Index(['primary_identifier', 'gender', 'network_stay',
       'average_monthly_bill_amount', 'dusage_sum', 'dusage_min', 'dusage_max',
       'dusage_avg', 'dusage_days', 'dusage_stddev', 'vusage_onnet_sum',
       'vusage_onnet_max', 'vusage_onnet_min', 'vusage_onnet_days',
       'vusage_onnet_avg', 'vusage_onnet_stddev', 'vusage_offnet_sum',
       'vusage_offnet_max', 'vusage_offnet_min', 'vusage_offnet_days',
       'vusage_offnet_avg', 'vusage_offnet_stddev',
       'number_of_fixed_bb_accounts', 'number_of_iptv_accounts',
       'add_on_tot_rental', 'add_on_count', 'device_type_2G', 'device_type_3G',
       'device_type_4G', 'device_category_Basic',
       'device_category_Feature phone', 'device_category_M2M',
       'device_category_Modem', 'device_category_Pluggable card',
       'device_category_Smartphone', 'device_category_Tablet',
       'district_name_Ampara', 'district_name_Anuradhapura',
       'district_name_Badulla', 'district_name_Batticaloa',
       'district_name_

In [ ]:
df2['device_category_M2M'] =0

In [ ]:
corrs = df2.corr()['next_month_plan'].abs()
columns = corrs[corrs > .05].index
corrs = corrs.filter(columns)
corrs

network_stay                     0.159052
average_monthly_bill_amount      0.594248
dusage_sum                       0.334217
dusage_min                       0.190001
dusage_max                       0.307844
dusage_avg                       0.331563
dusage_days                      0.202878
dusage_stddev                    0.310856
vusage_onnet_sum                 0.061139
vusage_onnet_max                 0.096796
vusage_onnet_days                0.055252
vusage_onnet_avg                 0.094965
vusage_onnet_stddev              0.093731
vusage_offnet_sum                0.068741
vusage_offnet_max                0.097726
vusage_offnet_days               0.333444
vusage_offnet_stddev             0.090531
add_on_tot_rental                0.111980
next_month_plan                  1.000000
device_type_2G                   0.129453
device_type_3G                   0.070329
device_type_4G                   0.150292
device_category_Basic            0.075868
device_category_Feature phone    0

In [ ]:
extracted_df = df2[['network_stay'                    ,
'average_monthly_bill_amount'  ,
'dusage_days'  ,
'dusage_stddev'  ,
'vusage_onnet_sum'  ,
'vusage_onnet_max'  ,
'vusage_onnet_days'  ,
'vusage_onnet_avg'  ,
'vusage_onnet_stddev'  ,
'vusage_offnet_sum'  ,
'vusage_offnet_max'  ,
'vusage_offnet_days'  ,
'vusage_offnet_stddev'  ,
'add_on_tot_rental'  ,
'next_month_plan'  ,
'device_type_2G'  ,
'device_type_3G'  ,
'device_type_4G'  ,
'device_category_Basic'  ,
'device_category_Feature phone'  ,
'device_category_Smartphone'  ,
'district_name_Colombo'  ,
'district_name_Jaffna'  ,
'age_group_30-40'    ]]     

extracted_df_test = df2_test[['primary_identifier',
    'network_stay'                    ,
'average_monthly_bill_amount'  ,
'dusage_days'  ,
'dusage_stddev'  ,
'vusage_onnet_sum'  ,
'vusage_onnet_max'  ,
'vusage_onnet_days'  ,
'vusage_onnet_avg'  ,
'vusage_onnet_stddev'  ,
'vusage_offnet_sum'  ,
'vusage_offnet_max'  ,
'vusage_offnet_days'  ,
'vusage_offnet_stddev'  ,
'add_on_tot_rental'  ,
'device_type_2G'  ,
'device_type_3G'  ,
'device_type_4G'  ,
'device_category_Basic'  ,
'device_category_Feature phone'  ,
'device_category_Smartphone'  ,
'district_name_Colombo'  ,
'district_name_Jaffna'  ,
'age_group_30-40'    ]]     

In [ ]:
extracted_df.shape

(10500, 24)

In [ ]:
extracted_df.to_csv("train.csv")
extracted_df_test.to_csv("test.csv")

In [ ]:
df2.to_csv("train_preprocessed.csv")
df2_test.to_csv("test_preprocessed.csv")

# Backup Network

In [ ]:

# import torch.optim as optim
# import matplotlib.pyplot as plt

# class NeuralNet(nn.Module):
#     def __init__(self, input_size, hidden1_size,hidden2_size, num_classes):
#         super(NeuralNet, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden1_size) 
#         self.relu = nn.ReLU()
#         self.fc2 = nn.Linear(hidden1_size, hidden2_size)  
#         self.relu = nn.ReLU()
#         self.fc3 = nn.Linear(hidden2_size,num_classes)
#         self.soft = nn.Softmax()
    
#     def forward(self, x):
#         out = self.fc1(x)
#         out = self.relu(out)
#         out = self.fc2(out)
#         out = self.relu(out)
#         out = self.fc3(out)
#         out = self.soft(out)
#         return out

# model = NeuralNet(109, 52,26, 8)
# # Define the loss
# criterion = nn.CrossEntropyLoss()
# # Optimizers require the parameters to optimize and a learning rate
# optimizer = optim.Adam(model.parameters(), lr=0.01,betas=(0.9, 0.999))
# epochs = 5
# limit = len(X_train)
# running_loss = []
# for e in range(epochs):
#     for i, x in zip(range(limit), X_train):
#         x = x.float()
#         x = x.view(-1,x.shape[0])
#         # Training pass
#         optimizer.zero_grad()
#         y = y_train.view(y_train.shape[0],8)
#         # y = y.type(torch.LongTensor)
#         label = y[i].view(1,8)
#         # print(label)
#         # label = torch.full((1,1), y[i], dtype=torch.float)
#         label = label.float()
#         output = model(x)
#         # print(output)
#         # print(label,output.detach().round())
#         # print(output.shape,label.shape)
#         # loss = criterion(output, label)
#         loss = criterion(output, torch.max(label, 1)[1])
#         # print(torch.max(label, 1)[1])
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         running_loss.append(loss.item())
#         if (i+1) % 500 == 0:
#             print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
#                    .format(e+1, epochs, i+1, limit,loss.item()))
#     # else:
#         # print(f"Training loss: {running_loss/limit}")
#     # print(e,running_loss)
#     # plt.plot(running_loss)

NameError: name 'X_train' is not defined